# text-classification-model-v1
This notebook classifies website data using transfer learning starting from an existing hugging face model
* Get a model checkpoint for an encoder model 
* Use reinforcement learning to apply the model on a new classification problem (EAGER website data) with limited new trained data
* Apply new head of model to full EAGER corpus to come up with mixes of models

## Install libraries

In [ ]:
# install bson for reading mongodb data
!pip uninstall --yes bson
!pip install pymongo

WARNING: Skipping bson as it is not installed. 
Requirement already satisfied: pymongo in /databricks/python3/lib/python3.8/site-packages (4.1.1)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

In [ ]:
# install hugging face 
!pip install transformers
!pip install datasets
!pip install DatasetDict

Requirement already satisfied: transformers in /databricks/python3/lib/python3.8/site-packages (4.11.3)
Requirement already satisfied: sacremoses in /databricks/python3/lib/python3.8/site-packages (from transformers) (0.0.46)
Requirement already satisfied: huggingface-hub>=0.0.17 in /databricks/python3/lib/python3.8/site-packages (from transformers) (0.5.1)
Requirement already satisfied: tqdm>=4.27 in /databricks/python3/lib/python3.8/site-packages (from transformers) (4.64.0)
Requirement already satisfied: numpy>=1.17 in /databricks/python3/lib/python3.8/site-packages (from transformers) (1.22.3)
Requirement already satisfied: pyyaml>=5.1 in /databricks/python3/lib/python3.8/site-packages (from transformers) (5.4.1)
Requirement already satisfied: regex!=2019.12.17 in /databricks/python3/lib/python3.8/site-packages (from transformers) (2021.4.4)
Requirement already satisfied: filelock in /databricks/python3/lib/python3.8/site-packages (from transformers) (3.0.12)
Requirement already satisfied: tokenizers<0.11,>=0.10.1 in /databricks/python3/lib/python3.8/site-packages (from transformers) (0.10.3)
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (from transformers) (2.25.1)
Requirement already satisfied: packaging>=20.0 in /databricks/python3/lib/python3.8/site-packages (from transformers) (20.9)
Requirement already satisfied: typing-extensions>=3.7.4.3 in /databricks/python3/lib/python3.8/site-packages (from huggingface-hub>=0.0.17->transformers) (3.7.4.3)
Requirement already satisfied: pyparsing>=2.0.2 in /databricks/python3/lib/python3.8/site-packages (from packaging>=20.0->transformers) (2.4.7)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests->transformers) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests->transformers) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests->transformers) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests->transformers) (2.10)
Requirement already satisfied: six in /databricks/python3/lib/python3.8/site-packages (from sacremoses->transformers) (1.15.0)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.8/site-packages (from sacremoses->transformers) (1.0.1)
Requirement already satisfied: click in /databricks/python3/lib/python3.8/site-packages (from sacremoses->transformers) (7.1.2)
 WARNING: You are using pip version 21.0.1; however, version 22.0.4 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command. 
Requirement already satisfied: datasets in /databricks/python3/lib/python3.8/site-packages (2.1.0)
Requirement already satisfied: xxhash in /databricks/python3/lib/python3.8/site-packages (from datasets) (3.0.0)
Requirement already satisfied: aiohttp in /databricks/python3/lib/python3.8/site-packages (from datasets) (3.8.1)
Requirement already satisfied: dill in /databricks/python3/lib/python3.8/site-packages (from datasets) (0.3.4)
Requirement already satisfied: fsspec[http]>=2021.05.0 in /databricks/python3/lib/python3.8/site-packages (from datasets) (2022.3.0)
Requirement already satisfied: numpy>=1.17 in /databricks/python3/lib/python3.8/site-packages (from datasets) (1.22.3)
Requirement already satisfied: responses<0.19 in /databricks/python3/lib/python3.8/site-packages (from datasets) (0.18.0)
Requirement already satisfied: multiprocess in /databricks/python3/lib/python3.8/site-packages (from datasets) (0.70.12.2)
Requirement already satisfied: huggingface-hub<1.0.0,>=0.1.0 in /databricks/python3/lib/python3.8/site-packages (from datasets) (0.5.1)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.8/site-packages (from d

In [ ]:
import tensorflow as tf
import numpy as np
from transformers import pipeline, AutoTokenizer, TFAutoModelForSequenceClassification

import gzip
import tarfile
import shutil
import bson
import pandas as pd

## Canned Examples

In [ ]:
print(pipeline('sentiment-analysis')('This application looks promising'))

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
[{'label': 'POSITIVE', 'score': 0.9940495491027832}]

In [ ]:
# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "This course is amazing!",
]
batch = dict(tokenizer(sequences, padding=True, truncation=True, return_tensors="tf"))

# This is new
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
labels = tf.convert_to_tensor([1, 1])
model.train_on_batch(batch, labels)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Out[5]: 15.002738952636719

## EAGER test

In [ ]:
# EAGER data -- unpack
tar_dir = '/dbfs/FileStore/eager/'
tar_file = tar_dir + "FirmDB_about2_20190131.tar"
print (tar_file)

/dbfs/FileStore/eager/FirmDB_about2_20190131.tar

In [ ]:
# tar = tarfile.open(tar_file)
# tar.extractall(tar_dir)
# tar.close()

In [ ]:
# gunzip about us pages
ungzip_file = tar_dir + "FirmDB_about2_20190131/pages_ABOUT2.bson"
gzip_file = ungzip_file + ".gz"
print (gzip_file)

/dbfs/FileStore/eager/FirmDB_about2_20190131/pages_ABOUT2.bson.gz

In [ ]:
with gzip.open(gzip_file, 'rb') as f_in:
    with open(ungzip_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
with open(ungzip_file,'rb') as f:
    about_pages = bson.decode_all(f.read())

In [ ]:
about_pages[0]['full_text']

Out[11]: ['About Xintec',
 'Introduction',
 'Vision & Core Value',
 'Management Team',
 'Milestone',
 'Press Center',
 'Quality & ESH Policy',
 'Business Ethics',
 'Contact Us',
 'Products / Services',
 'Optical Sensor CSP',
 'PPI & 3D Chip Scale Package',
 'MEMS & Sensor Packaging',
 'Design and Modeling Services',
 'Investor Relation',
 'Company Profile',
 'Monthly Revenue',
 'Quarterly Results',
 'Financial Reports',
 "Shareholders' Service",
 'Material Information',
 'TSE MOPS',
 'Calendar of Events',
 'FAQ',
 'Human Resources',
 'Training & Development',
 'Work & Life',
 'Recruting',
 'CSR',
 'Message from Chairman',
 'Corporate Governance',
 'Stakeholder Engagement',
 'Work Environment & Social Contribution',
 'CSR Policy and Reports',
 '繁中',
 'About Xintec',
 '\r\n >\r\n ',
 'About Xintec',
 '\r\n > Vision & Core Value\r\n ',
 'Introduction',
 'Vision & Core Value',
 'Management Team',
 'Milestone',
 'Press Center',
 'Quality & ESH Policy',
 'Business Ethics',
 'Contact Us',
 'Vision & Core Value',
 'Company Vision',
 'Be the best advanced wafer level packaging foundry service provider.',
 'Company Core Value',
 'Integrity: Being honest to customers, shareholder and ourselves. Willing to commit and always fulfill commitment.',
 'Innovation: Passionately generate and realize value adding ideas.',
 'Customer Orientation: Offer customer desired technology, robust production, excellent quality and premium service.',
 'Copyright © 2014 Xintec Inc. ALL rights reserved.\r\n \xa0|\xa0\r\n ',
 'Site Map',
 '\r\n \xa0|\xa0\r\n ',
 'Contact Us',
 '\r\n This website is best viewed by Internet Explorer 8+']

### Zero-shot classification

In [ ]:
firm_file_location = '/dbfs/FileStore/eager/about/Zygo.txt'
txt_file = open(firm_file_location, "r")
content_list = txt_file.readlines()
print(content_list)

['Metrology\n', '3Optical Profilers\n', 'Laser Interferometers\n', 'Position Metrology Systems\n', 'Custom Metrology Solutions\n', 'Trade-in & Upgrade Program\n', 'Optics\n', 'Optical Components Group\n', 'Electro-Optics Group\n', '- Engineering & Product Development\n', '- Electro-Optical Assembly\n', '- Opto-Mechanical Assembly\n', '- Turnkey System Manufacturing\n', 'Markets & Applications\n', '[close]\n', '\n', 'By continuing to use this site, you agree to our \n', 'Privacy and Cookie Policy\n', '. \n', 'OK\n', "What's New at ZYGO?\n", '››\n', 'News:\n', ' Launches Latest Generation of ZeGage Non-Contact 3Optical Profilers\n', '››\n', 'Blog:\n', ' Using Grazing Incidence Interferometry to Measure Large or Non-Specular Surfaces\n', '››\n', 'Blog:\n', ' Process Metrology Helps Ensure Quality of Smartphone Camera Lenses\n', '››\n', 'Blog:\n', ' Precision Lead Angle Metrology Improves Shaft Seals and Engine Efficiency\n', '››\n', 'Blog:\n', ' Precision Gear Tooth Metrology Helps Automaker Silence Humming Differential\n', '››\n', 'Blog:\n', ' Measuring Roughness on Super-Smooth Surfaces\n', '››\n', 'Blog:\n', ' New Smart Setup Feature Streamlines Precision Surface Metrology\n', '››\n', 'Blog:\n', ' Think you know ZYGO? Check out these six surprising facts and statistics...\n', 'Visit at these\n', 'Upcoming Events:\n', 'MD&West\n', '02/05\n', 'Photonics West\n', '02/05\n', 'Advanced Lithography\n', '02/26\n', 'Houstex\n', '02/26\n', 'Defense & Sensing\n', '04/16\n', 'Ann. Mtg. & Exhibt\n', '05/19\n', 'West\n', '07/09\n', 'Optics & Photonics\n', '08/13\n', 'Optifab\n', '10/15\n', '(Click for a complete list)\n', 'Privacy and Cookie Policy\n', 'Privacy Policy Inquiry Form\n', 'Unsubscribe\n', '×\n', 'Privacy Policy InquiryAbout Us\n', 'Company Profile\n', 'News Releases\n', 'Trade Show Schedule\n', 'Manufacturing Locations\n', 'Sales/Service Offices\n', 'How to Get to Patent Information\n', 'Registrations\n', 'Conflict Minerals Policy\n', 'Privacy and Cookie Policy\n', 'Terms & Conditions\n', "ZYGO's booth at Photonics West\n", 'Micro-Seminar in Progress\n', 'free\n', '~ 2019 ~\n', 'MD&West\n', 'Photonics West 2019 Exhibition\n', 'W3 Fair+Convention 2019\n', 'Advanced Lithography\n', 'Houstex\n', 'Defense & Sensing\n', 'CONTROL\n', 'Annual Meeting & Exhibition\n', 'EUSPEN\n', 'LASER\n', 'West\n', 'Optics & Photonics\n', 'Optifab 2019\n', 'IMTS\n', '[close]\n', '\n', 'By continuing to use this site, you agree to our \n', 'Privacy and Cookie Policy\n', '. \n', 'OK\n', "What's New at ZYGO?\n", '››\n', 'News:\n', ' Launches Latest Generation of ZeGage Non-Contact 3Optical Profilers\n', '››\n', 'Blog:\n', ' Using Grazing Incidence Interferometry to Measure Large or Non-Specular Surfaces\n', '››\n', 'Blog:\n', ' Process Metrology Helps Ensure Quality of Smartphone Camera Lenses\n', '››\n', 'Blog:\n', ' Precision Lead Angle Metrology Improves Shaft Seals and Engine Efficiency\n', '››\n', 'Blog:\n', ' Precision Gear Tooth Metrology Helps Automaker Silence Humming Differential\n', '››\n', 'Blog:\n', ' Measuring Roughness on Super-Smooth Surfaces\n', '››\n', 'Blog:\n', ' New Smart Setup Feature Streamlines Precision Surface Metrology\n', '››\n', 'Blog:\n', ' Think you know ZYGO? Check out these six surprising facts and statistics...\n', 'Visit at these\n', 'Upcoming Events:\n', 'MD&West\n', '02/05\n', 'Photonics West\n', '02/05\n', 'Advanced Lithography\n', '02/26\n', 'Houstex\n', '02/26\n', 'Defense & Sensing\n', '04/16\n', 'Ann. Mtg. & Exhibt\n', '05/19\n', 'West\n', '07/09\n', 'Optics & Photonics\n', '08/13\n', 'Optifab\n', '10/15\n', '(Click for a complete list)\n', 'Privacy and Cookie Policy\n', 'Privacy Policy Inquiry Form\n', 'Unsubscribe\n', '×\n', 'Privacy Policy InquiryOptical Surface Profilers\n', 'Products\n', '3Optical Profilers\n', 'Laser Interferometers\n', 'Position Metrology Systems\n', 'Custom Metrology Solutions\n', 'Trade-in & Upgrade Program\n', 'Optical Surface Profilers\n', 'ZeGage™ Pro\xa0 \n', 'New!\n', 'NewView™ 90

In [ ]:
content_list[6]

Out[13]: 'Optics\n'

In [ ]:
classifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)
Downloading: 0%| | 0.00/1.13k [00:00<?, ?B/s]
Downloading: 0%| | 0.00/1.52G [00:00<?, ?B/s]
Downloading: 0%| | 0.00/26.0 [00:00<?, ?B/s]
Downloading: 0%| | 0.00/878k [00:00<?, ?B/s]
Downloading: 0%| | 0.00/446k [00:00<?, ?B/s]
Downloading: 0%| | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
candidate_labels = ["business", "marketing", "manufacturing", "research", "engineering"]

for i in range(6,10):
  res = classifier(content_list[i], candidate_labels)
  print (res)

{'sequence': 'Optics\n', 'labels': ['business', 'marketing', 'research', 'engineering', 'manufacturing'], 'scores': [0.2477385550737381, 0.24553173780441284, 0.21116505563259125, 0.170455202460289, 0.12510938942432404]}
{'sequence': 'Optical Components Group\n', 'labels': ['business', 'engineering', 'research', 'manufacturing', 'marketing'], 'scores': [0.7122617959976196, 0.10993733257055283, 0.09078780561685562, 0.05439535900950432, 0.03261769190430641]}
{'sequence': 'Electro-Optics Group\n', 'labels': ['business', 'research', 'engineering', 'marketing', 'manufacturing'], 'scores': [0.6276072859764099, 0.19434507191181183, 0.10650159418582916, 0.03894878178834915, 0.03259727358818054]}
{'sequence': '- Engineering & Product Development\n', 'labels': ['engineering', 'business', 'research', 'manufacturing', 'marketing'], 'scores': [0.663918673992157, 0.17188161611557007, 0.11716794222593307, 0.036675307899713516, 0.010356508195400238]}

In [ ]:
classifier("I havern't a dog.")

Out[31]: [{'label': 'LABEL_0', 'score': 0.9532511234283447}]

In [ ]:
classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")

for i in range(0, len(content_list)):
  print ((content_list[i]))
  # res = classifier(content_list[i])
  # print (res)

Downloading: 0%| | 0.00/490 [00:00<?, ?B/s]
Downloading: 0%| | 0.00/255M [00:00<?, ?B/s]
Downloading: 0%| | 0.00/48.0 [00:00<?, ?B/s]
Downloading: 0%| | 0.00/226k [00:00<?, ?B/s]
Downloading: 0%| | 0.00/112 [00:00<?, ?B/s]
Metrology

3Optical Profilers

Laser Interferometers

Position Metrology Systems

Custom Metrology Solutions

Trade-in & Upgrade Program

Optics

Optical Components Group

Electro-Optics Group

- Engineering & Product Development

- Electro-Optical Assembly

- Opto-Mechanical Assembly

- Turnkey System Manufacturing

Markets & Applications

[close]



By continuing to use this site, you agree to our 

Privacy and Cookie Policy

. 

OK

What's New at ZYGO?

››

News:

 Launches Latest Generation of ZeGage Non-Contact 3Optical Profilers

››

Blog:

 Using Grazing Incidence Interferometry to Measure Large or Non-Specular Surfaces

››

Blog:

 Process Metrology Helps Ensure Quality of Smartphone Camera Lenses

››

Blog:

 Precision Lead Angle Metrology Improves Shaft Seals and Engine Efficiency

››

Blog:

 Precision Gear Tooth Metrology Helps Automaker Silence Humming Differential

››

Blog:

 Measuring Roughness on Super-Smooth Surfaces

››

Blog:

 New Smart Setup Feature Streamlines Precision Surface Metrology

››

Blog:

 Think you know ZYGO? Check out these six surprising facts and statistics...

Visit at these

Upcoming Events:

MD&West

02/05

Photonics West

02/05

Advanced Lithography

02/26

Houstex

02/26

Defense & Sensing

04/16

Ann. Mtg. & Exhibt

05/19

West

07/09

Optics & Photonics

08/13

Optifab

10/15

(Click for a complete list)

Privacy and Cookie Policy

Privacy Policy Inquiry Form

Unsubscribe

×

Privacy Policy InquiryAbout Us

Company Profile

News Releases

Trade Show Schedule

Manufacturing Locations

Sales/Service Offices

How to Get to Patent Information

Registrations

Conflict Minerals Policy

Privacy and Cookie Policy

Terms & Conditions

ZYGO's booth at Photonics West

Micro-Seminar in Progress

free

~ 2019 ~

MD&West

Photonics West 2019 Exhibition

W3 Fair+Convention 2019

Advanced Lithography

Houstex

Defense & Sensing

CONTROL

Annual Meeting & Exhibition

EUSPEN

LASER

West

Optics & Photonics

Optifab 2019

IMTS

[close]



By continuing to use this site, you agree to our 

Privacy and Cookie Policy

. 

OK

What's New at ZYGO?

››

News:

 Launches Latest Generation of ZeGage Non-Contact 3Optical Profilers

››

Blog:

 Using Grazing Incidence Interferometry to Measure Large or Non-Specular Surfaces

››

Blog:

 Process Metrology Helps Ensure Quality of Smartphone Camera Lenses

››

Blog:

 Precision Lead Angle Metrology Improves Shaft Seals and Engine Efficiency

››

Blog:

 Precision Gear Tooth Metrology Helps Automaker Silence Humming Differential

››

Blog:

 Measuring Roughness on Super-Smooth Surfaces

››

Blog:

 New Smart Setup Feature Streamlines Precision Surface Metrology

››

Blog:

 Think you know ZYGO? Check out these six surprising facts and statistics...

Visit at these

Upcoming Events:

MD&West

02/05

Photonics West

02/05

Advanced Lithography

02/26

Houstex

02/26

Defense & Sensing

04/16

Ann. Mtg. & Exhibt

05/19

West

07/09

Optics & Photonics

08/13

Optifab

10/15

(Click for a complete list)

Privacy and Cookie Policy

Privacy Policy Inquiry Form

Unsubscribe

×

Privacy Policy InquiryOptical Surface Profilers

Products

3Optical Profilers

Laser Interferometers

Position Metrology Systems

Custom Metrology Solutions

Trade-in & Upgrade Program

Optical Surface Profilers

ZeGage™ Pro  

New!

NewView™ 9000

Nexview™NX2

Nomad™

Compass™

APM650™

Mx™ Software

Profiler Objectives

Optical Profiler Basics

Surface Roughness Measurement

Surface Profiling

3Optical Microscopy

3Interference Microscopy

ZYGO's 3Optical Profiler instruments enable precise, quantitative, ISO-compliant, 

non-contact

 surface measurement and characterization of micro- and nano-scale surface features, capturing 

up to two million data points

in just seconds


### Garbage classifier
keep test == 1, discard == 0

In [ ]:
import datasets
from sklearn.model_selection import train_test_split
import pandas as pd

In [ ]:
firm_file_location = '/dbfs/FileStore/eager/garbage_classifier_input.csv'
input_df = pd.read_csv(firm_file_location)
input_df[input_df['sample_text'].isnull()]

Out[7]:

,sample_text,keep_text


In [ ]:
dataset = datasets.load_dataset('csv', data_files=firm_file_location,  split='train')
dataset.cast_column("keep_text", datasets.Value('int8'))

Using custom data configuration default-f17b8338c5a1d3a3
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-f17b8338c5a1d3a3/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
Casting the dataset: 0%| | 0/1 [00:00<?, ?ba/s]
Out[11]: Dataset({
 features: ['sample_text', 'keep_text'],
 num_rows: 1546
})

In [ ]:
# 80% train, 20% test + validation
train_test_dataset = dataset.train_test_split(test_size=0.1)
# Split the 10% test + valid in half test, half valid
test_valid_dataset = train_test_dataset['test'].train_test_split(test_size=0.2)
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = datasets.DatasetDict({
    'train': train_test_dataset['train'],
    'test': test_valid_dataset['test'],
    'valid': test_valid_dataset['train']})

In [ ]:
train_test_valid_dataset

Out[39]: DatasetDict({
 train: Dataset({
 features: ['sample_text', 'keep_text'],
 num_rows: 1391
 })
 test: Dataset({
 features: ['sample_text', 'keep_text'],
 num_rows: 31
 })
 valid: Dataset({
 features: ['sample_text', 'keep_text'],
 num_rows: 124
 })
})

In [ ]:
def tokenize_function(x):
  return tokenizer(x["sample_text"], truncation=True, max_length=100)

In [ ]:
tokenized_dataset = train_test_valid_dataset.map(tokenize_function, batched=True, batch_size=2000)

0%| | 0/1 [00:00<?, ?ba/s]
 0%| | 0/1 [00:00<?, ?ba/s]
 0%| | 0/1 [00:00<?, ?ba/s]

In [ ]:
samples = tokenized_dataset["train"].to_dict()
samples = {k: v for k, v in samples.items() if k not in ["idx", "sample_text"]}
set([len(x) for x in samples["input_ids"]])

Out[42]: {2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 74,
 75,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 89,
 90,
 91,
 92,
 94,
 95,
 96,
 99,
 100}

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="max_length", max_length=100, return_tensors="tf")

In [ ]:
batch = data_collator(samples)
{k: v.shape for k, v in batch.items()}

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 /databricks/python/lib/python3.8/site-packages/transformers/tokenization_utils_base.py in convert_to_tensors (self, tensor_type, prepend_batch_axis) 
 704 if not is_tensor ( value ) : 
 --> 705 tensor = as_tensor ( value ) 
 706 

 /databricks/python/lib/python3.8/site-packages/tensorflow/python/framework/constant_op.py in constant (value, dtype, shape, name) 
 266 """
 --> 267 return _constant_impl(value, dtype, shape, name, verify_shape=False,
 268 allow_broadcast=True)

 /databricks/python/lib/python3.8/site-packages/tensorflow/python/framework/constant_op.py in _constant_impl (value, dtype, shape, name, verify_shape, allow_broadcast) 
 278 return _constant_eager_impl ( ctx , value , dtype , shape , verify_shape ) 
 --> 279 return _constant_eager_impl ( ctx , value , dtype , shape , verify_shape ) 
 280 

 /databricks/python/lib/python3.8/site-packages/tensorflow/python/framework/constant_op.py in _constant_eager_impl (ctx, value, dtype, shape, verify_shape) 
 303 """Creates a constant on the current device.""" 
 --> 304 t = convert_to_eager_tensor ( value , ctx , dtype ) 
 305 if shape is None : 

 /databricks/python/lib/python3.8/site-packages/tensorflow/python/framework/constant_op.py in convert_to_eager_tensor (value, ctx, dtype) 
 101 ctx . ensure_initialized ( ) 
 --> 102 return ops . EagerTensor ( value , ctx . device_name , dtype ) 
 103 

 ValueError : Can't convert Python sequence with mixed types to Tensor.

During handling of the above exception, another exception occurred:

 ValueError Traceback (most recent call last)
 <command-1849903833854631> in <module> 
 ----> 1 batch = data_collator ( samples ) 
 2 { k : v . shape for k , v in batch . items ( ) } 

 /databricks/python/lib/python3.8/site-packages/transformers/data/data_collator.py in __call__ (self, features) 
 219 
 220 def __call__ ( self , features : List [ Dict [ str , Any ] ] ) -> Dict [ str , Any ] : 
 --> 221 batch = self.tokenizer.pad(
 222 features , 
 223 padding = self . padding , 

 /databricks/python/lib/python3.8/site-packages/transformers/tokenization_utils_base.py in pad (self, encoded_inputs, padding, max_length, pad_to_multiple_of, return_attention_mask, return_tensors, verbose) 
 2793 batch_outputs [ key ] . append ( value ) 
 2794 
 -> 2795 return BatchEncoding ( batch_outputs , tensor_type = return_tensors ) 
 2796 
 2797 def create_token_type_ids_from_sequences(

 /databricks/python/lib/python3.8/site-packages/transformers/tokenization_utils_base.py in __init__ (self, data, encoding, tensor_type, prepend_batch_axis, n_sequences) 
 208 self . _n_sequences = n_sequences
 209 
 --> 210 self . convert_to_tensors ( tensor_type = tensor_type , prepend_batch_axis = prepend_batch_axis ) 
 211 
 212 @ property

 /databricks/python/lib/python3.8/site-packages/transformers/tokenization_utils_base.py in convert_to_tensors (self, tensor_type, prepend_batch_axis) 
 719 "Please see if a fast version of this tokenizer is available to have this feature available." 
 720 )
 --> 721 raise ValueError(
 722 "Unable to create tensor, you should probably activate truncation and/or padding " 
 723 "with 'padding=True' 'truncation=True' to have batched tensors with the same length." 

 ValueError : Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length.

In [ ]:
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols="keep_text",
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_dataset["valid"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols="keep_text",
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:
tf_train_dataset

Out[46]: <PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(8, None), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(8, None), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(8, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(8,), dtype=tf.float32, name=None))>

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

1/154 [..............................] - ETA: 35:46 - loss: 0.8336 - accuracy: 0.2500 2/154 [..............................] - ETA: 13s - loss: 0.7732 - accuracy: 0.4375  3/154 [..............................] - ETA: 13s - loss: 0.7331 - accuracy: 0.5000 4/154 [..............................] - ETA: 12s - loss: 0.7551 - accuracy: 0.5000 5/154 [..............................] - ETA: 12s - loss: 1.1980 - accuracy: 0.4000 6/154 [>.............................] - ETA: 12s - loss: 1.0957 - accuracy: 0.4583 7/154 [>.............................] - ETA: 12s - loss: 1.0343 - accuracy: 0.5000 8/154 [>.............................] - ETA: 12s - loss: 0.9836 - accuracy: 0.5312 9/154 [>.............................] - ETA: 12s - loss: 0.9508 - accuracy: 0.5278 10/154 [>.............................] - ETA: 12s - loss: 0.9137 - accuracy: 0.5500 11/154 [=>............................] - ETA: 12s - loss: 0.9678 - accuracy: 0.5227 12/154 [=>............................] - ETA: 11s - loss: 1.0156 - accuracy: 0.5104 13/154 [=>............................] - ETA: 11s - loss: 1.0035 - accuracy: 0.5096 14/154 [=>............................] - ETA: 11s - loss: 1.0273 - accuracy: 0.4821 15/154 [=>............................] - ETA: 11s - loss: 1.0009 - accuracy: 0.4917 16/154 [==>...........................] - ETA: 11s - loss: 0.9629 - accuracy: 0.5156 17/154 [==>...........................] - ETA: 11s - loss: 0.9279 - accuracy: 0.5368 18/154 [==>...........................] - ETA: 11s - loss: 0.9429 - accuracy: 0.5347 19/154 [==>...........................] - ETA: 11s - loss: 0.9260 - accuracy: 0.5461 20/154 [==>...........................] - ETA: 11s - loss: 0.8988 - accuracy: 0.5625 21/154 [===>..........................] - ETA: 11s - loss: 0.8802 - accuracy: 0.5714 22/154 [===>..........................] - ETA: 11s - loss: 0.8690 - accuracy: 0.5739 23/154 [===>..........................] - ETA: 10s - loss: 0.8538 - accuracy: 0.5870 24/154 [===>..........................] - ETA: 10s - loss: 0.8444 - acc

In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 8
num_epochs = 5
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

In [ ]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=num_epochs)

Epoch 1/5
 1/173 [..............................] - ETA: 39:34 - loss: 0.8536 - accuracy: 0.2500 2/173 [..............................] - ETA: 14s - loss: 0.7725 - accuracy: 0.3750  3/173 [..............................] - ETA: 14s - loss: 0.7212 - accuracy: 0.5833 4/173 [..............................] - ETA: 14s - loss: 0.7080 - accuracy: 0.5938 5/173 [..............................] - ETA: 14s - loss: 0.6895 - accuracy: 0.6000 6/173 [>.............................] - ETA: 13s - loss: 0.6510 - accuracy: 0.6458 7/173 [>.............................] - ETA: 13s - loss: 0.6435 - accuracy: 0.6429 8/173 [>.............................] - ETA: 13s - loss: 0.6344 - accuracy: 0.6562 9/173 [>.............................] - ETA: 13s - loss: 0.6341 - accuracy: 0.6528 10/173 [>.............................] - ETA: 13s - loss: 0.6575 - accuracy: 0.6250 11/173 [>.............................] - ETA: 13s - loss: 0.6285 - accuracy: 0.6591 12/173 [=>............................] - ETA: 13s - loss: 0.6176 - accuracy: 0.6667 13/173 [=>............................] - ETA: 13s - loss: 0.6229 - accuracy: 0.6538 14/173 [=>............................] - ETA: 13s - loss: 0.6288 - accuracy: 0.6429 15/173 [=>............................] - ETA: 13s - loss: 0.6054 - accuracy: 0.6667 16/173 [=>............................] - ETA: 13s - loss: 0.5871 - accuracy: 0.6875 17/173 [=>............................] - ETA: 13s - loss: 0.5845 - accuracy: 0.6912 18/173 [==>...........................] - ETA: 12s - loss: 0.5772 - accuracy: 0.6944 19/173 [==>...........................] - ETA: 12s - loss: 0.5679 - accuracy: 0.7039 20/173 [==>...........................] - ETA: 12s - loss: 0.5597 - accuracy: 0.7125 21/173 [==>...........................] - ETA: 12s - loss: 0.5560 - accuracy: 0.7143 22/173 [==>...........................] - ETA: 12s - loss: 0.5386 - accuracy: 0.7273 23/173 [==>...........................] - ETA: 12s - loss: 0.5300 - accuracy: 0.7337 24/173 [===>..........................] - ETA: 12s - loss: 